In [1]:
import requests, re, jmespath
from pandas import DataFrame

In [ ]:
# find api key in google drive
API_KEY = ""

baseURL = "https://api.scripture.api.bible/v1"
headers = {"api-key": API_KEY}

In [ ]:
def pipeline(bibleID) -> DataFrame:
    allVerses = []

    # request bible books with chapter info
    url = f"{baseURL}/bibles/{bibleID}/books"
    response = requests.get(url, headers=headers, params={"include-chapters": True}).json()

    # extract chapter IDs excluding intro chapters
    chapIDs = jmespath.search("data[].chapters[?number!='intro'].id[]", response)

    # get verses for each chapter
    for chapID in chapIDs:

        # request chapter info
        url = f"{baseURL}/bibles/{bibleID}/chapters/{chapID}"
        response = requests.get(url, headers=headers, params={"content-type": "text"}).json()
        print(response)

        # str of all verses in the chapter
        verses = response["data"]["content"]

        # split into individual verses
        verseList = re.split(r"\s*\[\d+\]\s*", verses.strip())
        verseList = [v.strip() for v in verseList if v]

        # make an entry for each verse
        for i, verse in enumerate(verseList):
            allVerses.append({"citation": f"{response['data']['id']}.{i+1}", "verse": verse})

    return DataFrame(allVerses)

### make csv for all the bibles
citation, verse\
GEN.1.1, In the begin...

In [ ]:
bibleIDs = {
    "ASV": "06125adad2d5898a-01",
    "WEB": "72f4e6dc683324df-01",
    "FBV": "65eec8e0b60e656b-01",
    "WMB": "f72b840c855f362c-04",
    "KJV": "de4e12af7f28f599-02",
}

In [ ]:
asv = pipeline(bibleIDs["ASV"])
asv.to_csv("asv.csv", index=False)

In [ ]:
web = pipeline(bibleIDs["WEB"])
web.to_csv("web.csv", index=False)

In [ ]:
wmb = pipeline(bibleIDs["WMB"])
wmb.to_csv("wmb.csv", index=False)

In [ ]:
fbv = pipeline(bibleIDs["FBV"])
fbv.to_csv("fbv.csv", index=False)

In [ ]:
kjv = pipeline(bibleIDs["KJV"])
kjv.to_csv("kjv.csv", index=False)